In [5]:
import pandas as pd
import numpy as np
import json
from pandas.api.types import CategoricalDtype

In [82]:
with open('//home//kin672//Kira_GenTB//Isolating Relevant Files//relevant_file_names_v2.json') as f:
    relevant_files = json.load(f) # This is a dictionary with format {STRAIN ID: {Folder: , Resistance, Lineage}


len(relevant_files)

8137

In [84]:
prefix = '//n//groups//gentb_www//predictData//'

# RF probability thresholds (if probability is below threshold, it is susceptible to the drug)
thresholds = {'rif': 0.002, 'inh': 0.22, 'emb': 0.082,'pza': 0.013,'str':0.047, 'cap':0.25,'amk':0.6,'cip': 0.42,'kan': 0.63,'levo':0.41,'oflx':0.33,'pas': 0.001,'eth': 0.32}
relevant_files = {
    "3213329_IonXpress_020": {
        "Folder": "tbdata_00001241",
        "Resistance": "3213329_IonXpress_020.matrix.json",
        "Lineage": "See array"
    },
    "3212705_IonXpress_024": {
        "Folder": "tbdata_00001241",
        "Resistance": "3212705_IonXpress_024.matrix.json",
        "Lineage": "See array"
    },
    "TEST0248": {
        "Folder": "tbdata_00001312",
        "Resistance": "TEST0248.matrix.json",
        "Lineage": "//n//groups//gentb_www//predictData//tbdata_00001312//TEST0248_cut_lineage.txt"
    },
    "TEST0246": {
        "Folder": "tbdata_00001312",
        "Resistance": "TEST0246.matrix.json",
        "Lineage": "//n//groups//gentb_www//predictData//tbdata_00001312//TEST0246_cut_lineage.txt"
    }
}

lineage_table = pd.read_csv('/home/kin672/Kira_GenTB/Isolating Relevant Files/vcf_to_lineages.csv')
lineage_table

,Isolate,Lineage
0,3213167_IonXpress_029,4.9(1/1)
1,3213540_IonXpress_032,4.9(1/1)
2,3213169_IonXpress_030,"4.3(1/1),4.9(1/1)"
3,3213034_IonXpress_026,3(1/1)
4,3211622_IonXpress_017,1.2.2(1/1)
...,...,...
5618,TB-111-16668-18-138_S124_R.all,2.2.1(1/1)
5619,TB-111-16552-18_S32_R.all,4.2.1(1/1)
5620,TB-111-17896-18_S97_R.all,4.8(1/1)
5621,TB-111-19509-18_S167_R.all,4.8(1/1)


In [88]:
x = lineage_table.loc[:, 'Lineage'].str.split(',', expand=True)


0                4.9(1/1)
1                4.9(1/1)
2       4.3(1/1),4.9(1/1)
3                  3(1/1)
4              1.2.2(1/1)
              ...        
5618           2.2.1(1/1)
5619           4.2.1(1/1)
5620             4.8(1/1)
5621             4.8(1/1)
5622             4.8(1/1)
Name: Lineage, Length: 5623, dtype: object

In [86]:
lin_table = pd.concat([lineage_table[['Isolate']], x.loc[:, 0:2]], axis = 1)
lin_table.to_csv('//home//kin672//Kira_GenTB//Isolating Relevant Files//vcf_to_lineages-sep.csv', index = False)


In [122]:
catch_problems2 = []
catch_problems = []
catch_problems_3 = []
master_df = pd.DataFrame({'ID':[], 'Drug':[], 'Resistant':[], 'Lineage':[]})

for strainID in relevant_files:
    value = relevant_files.get(strainID)
    json_path = prefix + value.get('Folder') + '//' + value.get('Resistance')
    lineage_path = value.get('Lineage')
        
    # Parse lineage
    if lineage_path == 'See array':
        freschi_lineage = lineage_table.loc[lineage_table.Isolate == strainID, 'Lineage'].to_list()
        freschi_lineage = [str(i).replace('(1/1)', '') for i in freschi_lineage]
        freschi_lineage = ', '.join(freschi_lineage)
    else:
        try:
            lineage = pd.read_csv(lineage_path, sep = '\t')
        except:
            catch_problems_3.append(lineage_path)
            continue
        if any(['freschi' in string for string in lineage.columns]):
            freschi_lineage = lineage.loc[0, list(lineage.columns)[list(np.where(['freschi' in string for string in lineage.columns])[0])[0]]].replace('(1/1)','') 
        else:
            catch_problems_3.append(lineage_path)
            continue
    
    # Parse json
    with open(json_path) as f:
        resistance = json.load(f)
        
    # Cut out the extra output at the bottom
    resistance = resistance[0]
    if len(resistance) != 13:
        catch_problems2.append(json_path)
    
    # Pull binary resistance outcome and probability per drug!
    for drug_index in range(len(resistance)):
        profile = resistance[drug_index]
        drug_name = profile[1]
        resistant = 'unfilled'
        
        # If the json file has only three outputs per drug, we need to manually determine the binary output using thresholds.
        if len(profile) == 5: # 3 outputs - probability is at index 2
            if float(profile[2]) < thresholds[drug_name]:
                resistant = '0'
            else:
                resistant = '1'
        elif len(profile) == 6: # 4 outputs - binary output is at index 2
            resistant = profile[2] 
        else:
            catch_problems.append(strainID + '/' + drug_name)
            continue
        
        # Add a line to the dataframe
        master_df = pd.concat([master_df, pd.DataFrame({'ID':[strainID], 'Drug':[drug_name], 'Resistant':[resistant], 'Lineage':[freschi_lineage]})], ignore_index = True)
    
        



In [102]:
print('Total number of strains processed: ' + str(len(master_df) / 13) + '\n')
print('StrainIDs that were not processed: ' + ', '.join(catch_problems))
print('Json files without 13 drugs: ' + ', '.join(catch_problems2))
print('Lineage files without freschi2020: ' + ', '.join(catch_problems_3))


Total number of strains processed: 4.0

StrainIDs that were not processed: 
Json files without 13 drugs: 
Lineage files without freschi2020: 


In [123]:
master_df

,ID,Drug,Resistant,Lineage
0,3213329_IonXpress_020,inh,0,1.2.2
1,3213329_IonXpress_020,rif,0,1.2.2
2,3213329_IonXpress_020,pza,0,1.2.2
3,3213329_IonXpress_020,emb,0,1.2.2
4,3213329_IonXpress_020,str,0,1.2.2
5,3213329_IonXpress_020,eth,0,1.2.2
6,3213329_IonXpress_020,kan,0,1.2.2
7,3213329_IonXpress_020,cap,0,1.2.2
8,3213329_IonXpress_020,amk,0,1.2.2
9,3213329_IonXpress_020,cip,0,1.2.2


In [106]:
drug_order = CategoricalDtype(thresholds.keys(), ordered = True)

In [107]:
master_df['Drug'] = master_df['Drug'].astype(drug_order)
master_df = master_df.sort_values(by = ['Drug', 'Lineage','Resistant'])

In [108]:
summarized = master_df.groupby(['Drug','Lineage','Resistant']).count()
summarized = summarized.rename(columns={'ID':'NumStrains'})
summarized.reset_index(inplace=True)

In [109]:
reformat_summarized = pd.DataFrame({'Drug':[], 'Lineage':[], 'Percentage of Strains that are Resistant':[], 'Number of Resistant Strains':[], 'Number of Susceptible Strains':[]})
for drug in np.unique(summarized.Drug):
    for lineage in np.unique(summarized.Lineage):
        num_resistant = int(summarized[(summarized.Drug == drug) & (summarized.Lineage == lineage) & (summarized.Resistant == '1')].NumStrains)
        num_susceptible = int(summarized[(summarized.Drug == drug) & (summarized.Lineage == lineage) & (summarized.Resistant == '0')].NumStrains)
        temp = pd.DataFrame({'Drug':[drug], 'Lineage':[lineage], 'Percentage of Strains that are Resistant':[num_resistant * 100 / (num_resistant + num_susceptible)], 'Number of Resistant Strains':[str(num_resistant)], 'Number of Susceptible Strains':[str(num_susceptible)]})
        reformat_summarized = pd.concat([reformat_summarized, temp], ignore_index=True)
reformat_summarized

,Drug,Lineage,Percentage of Strains that are Resistant,Number of Resistant Strains,Number of Susceptible Strains
0,amk,1.2.2,0.0,0,1
1,amk,3.1.1,100.0,1,0
2,amk,4.2.1.2.2.1.2,0.0,0,1
3,amk,4.9,0.0,0,1
4,cap,1.2.2,0.0,0,1
5,cap,3.1.1,100.0,1,0
6,cap,4.2.1.2.2.1.2,0.0,0,1
7,cap,4.9,0.0,0,1
8,cip,1.2.2,0.0,0,1
9,cip,3.1.1,100.0,1,0


In [ ]:
reformat_summarized.to_csv('//home//kin672//Kira_GenTB//summarized_full.csv')